# Veri Setinin Eksik Verilerden Temizlenmesi Çalışması

Veri temizleme çalışmasına hoşgeldiniz. Bu çalışmada , veri setlerinde bulunan eksik ve yanlış verilerin veri setinden nasıl temizleneceğini ve eksik veriler yerine nasıl veri ekleneceğini göreceğiz.

Bugünkü derste yapacaklarımızı şöyle bir özetleyelim,

1. -Öncelikle veri setimizi inceliyoruz
1. -Sonra veri setimizde eksik veri olup olmadığını tesbit ediyoruz.
1. -Eksik verilerimizin hangi sebeplerden kaynaklandığına bakacağız.
1. -Eksik verilerimizi veri setinden temizleyeceğiz.
1. -Eksik veriler yerine başka değerler ekleyeceğiz.


**1-Veri Setini İncelemek**
********
Öncelikle Python'da kullanacağımız kütüphaneyi ve veri setimizi notebook platformuna yüklüyoruz. Bugünkü çalışmamızda Amerikan Futbol oyunlarında meydana gelen olayların istatistiğini göstereceğim, bu çalışmada veri setine izin verilen San Francisco bölgesi veri setini kullanacağız.

In [1]:
# Kullanacağımız modül ve kütüphaneler
import pandas as pd
import numpy as np

# Verilerimizi ilgili veri setinden okuyoruz
nlf_data = pd.read_csv("../input/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# Tekrar işlemleri için (np.random.seed(0)) kodunu ekliyoruz.
np.random.seed(0) 

İlk olarak elimdeki veri setimizin ne olduğunu bakıyoruz, yani nasıl bir veri setimiz var öncelikle inceliyoruz. Bu işlem bize veri setimizin doğru birşekilde yüklenip yüklenmediği bilgisini verecektir.  
Şimdi elimdeki veri setinde eksik (NaN yada None) olan değerleri kontrol ediyoruz.

In [2]:
# Veri setimizin (nlf_data) ilk birkaç satırına bakarak eksik değerleri kontrol edebiliriz.
nlf_data.sample(5)

Evet, yukarıda özet olarak görüntüledimiğiz verilerde NaN(eksik) verilerin olduğunu görebiliyoruz. Peki sf_permits veri setine de baklım mı ?
Yep, it looks like there's some missing values. What about in the sf_permits dataset?

In [3]:
# sf_Permits veri setimiz içeriğini 5 satırlık bir özetine bakarak eksik veri olup olmadığını gözlemleyeceğiz.
sf_permits.sample(5)

**Şimdi veri setimizdeki eksik veri sayısına bakalım**
****
Evet, şimdi kaç tane eksik verimiz olduğuna bakalım ve her satırdaki eksi veri sayılarını tesbit edelim.


In [5]:
#Şimdi her kolonda kaç tane eksik değer var onlara bakalım
missing_values_count = nlf_data.isnull().sum()

#Şimdi de  ilk 10 sütundaki eksik değerlerin kaç tane olduğuna bakalım
missing_values_count[0:10]

Evet yukarıda görüleceği üzere birçok sütunda eksik veriler mevcut. Problemimizin çözümünde bize doğru verilerin hassaslığı konusunda  bu bilgiler bize yardımcı olacaktır.

In [7]:
#Toplam kaç tane eksik verimiz var ona bakalım
total_cells = np.product(nlf_data.shape)
total_missing = missing_values_count.sum()
#Eksik verilerin yüzde(%) olarak ne kadar olduğuna bakalım
(total_missing/total_cells) * 100

Oooooo, yaklaşık % 25 eksik veri var. Bir sonraki adımda eksik verilere daha yakından bakacağız ve onalara ne olup bittiğini anlamaya çalışacağız.

In [8]:
#Peki sf_permits veri setimizdeki eksik verilerin oranını merak etmiyor musunuz ? Hadi birde ona bakalım
total_cells = np.product(sf_permits.shape)
total_missing = missing_values_count.sum()
#Şimdi de % oranını görüntüleyelim
(total_cells / total_missing)*100

**Veriler neden eksik gelin bu konuyu inceleyelim**
Verilerin neden olmadığı konusu  veri bilimcilerin bu işle uğraşırken ilk karşılarına çıkan durumlardan biridir. Ve bu işlem eğer deneyime sahip değilseniz ki yeni başlayanlar için deneyim :-) olmaz, dolayısı ile yeni başlayanların bu konuda anlamaları gereken bir çok durum var.  Veri setini alıp direkt analizler yapabiliriz, bu bize bazı sonuçlarda verecektir ancak bu sonuçlar gerçeği ne kadar yansıtıyor ? Asıl anlaşılması gereken konu bu !  Elde edilen veriler ne kadar doğru ? Bunu anlamak için veri setimizin içeriğinin nelerden oluştuğu hakkında bilgi edinmemiz gerekiyor. bu durum için yukarıda veri setimizi nasıl inceleyeceğimizi ve eksik verileri nasıl tesbit edeceğimizi görmüştük. Şimdi  kendimize sormamız gereken soru şu 
>** Bu veriler eksik peki neden ? Acaba veriler kayıt edilmemiş mi yoksa bu veriler üretilmemiş mi ?**

Eğer eksik olan veriler gerçekten üretilmetiği için  veri setinde yoksa " mesala çocuk sahibi olmayan bir ailenin en büyük çocuğunun boyunun olmaması gibi" bir durum ise bu durum için eksik veri olmuş olabilir. Bu değerleri biz artık NaN(Not a Number- Uygun olmayan değerler) olarak düşünebiliriz. Bir diğer durum ise " eğer veriler kayıt edilmemiş" ise ne yapcağımızdır. Artık burada yapılması gereken işlem  diğer verilere bakarak yaklaşık olarak neler olabileceği hakkında yorumlar yapabiliriz. Şimdi bu problemin nasıl çözüleceğini ve bu verileri nasıl anlamlandıracağımızı göreceğiz.(Bu duruma "imputation-(töhmek-başkasına atma" denilmektedir)).
Gelin bir örnek üzerinde bu durumu incelemeye çalışalım. nlf_data verisetindeki veriler üzerinde bu işlemi 'TimeSecs' sütünunda inceleyelim, çünkü bu sütünda çok fazle eksik veri var.



In [9]:
#şimdi ilk 10 satırdaki eksik değerlere bakalım
missing_values_count[0:10]

Ekteki dökümana  [dökümana buradan ulaşabilirsiniz](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016) baktığımızda şunu gözlemleyebiliyoruz," Oyunun yapıldığı sırada bu sütunun oyunda kalan saniye sayısı hakkında bilgi verdiğini görebiliyoruz. " Bunun anlamı şu " veriler eksik çünkü bu değerler kayıt edilmemiş. Bu yüzden NA değerlerinin  neler olduğunu tahmin etmemiz gerekiyor ki doğru  sonuçlara yakın veriler elde edebilelim. Öte yandan, aynı zamanda kayıp alanlara sahip olan 'PenalizedTeam' gibi başka alanlar da var. Bu durumda, alan eksiktir, çünkü ceza olmazsa, * takımın cezalandırıldığını söylemek mantıklı değildir. Bu sütun için, ya boş bırakmak ya da "ne" gibi üçüncü bir değer eklemek daha mantıklı olacaktır ve NA'ları değiştirmek için bunu kullanın.

Öte yandan, aynı zamanda kayıp alanlara sahip olan 'PenalizedTeam' gibi başka alanlar da var. Bu alan eksiktir, çünkü ceza olmazsa,  * takımın cezalandırıldığını söylemek mantıklı değildir . Bu sütun için, ya boş bırakmak ya da "NA" gibi üçüncü bir değer eklemek daha mantıklı olacaktır ve  bu alanları NA'ları değiştirmek  gerekmektedir 
> **Ipucu** :  Eğer başka bir kişiden aldığınız verileri okuyorsanız, bu yöntem verileri okumak için iyi bir yol, ancak daha da iyi olanı verileri aldığınız kişiye ulaşıp verilerin tamamını almanız daha iyi olacaktır.

Eğer çok ciddi bir veri seti üzerinde çalışıyorsanız, bu durumlara çok dikkat etmemiz gerekiyor, veri setinin orjinaline bakarak eksik değerlerin neler ekleneceği ve nelerin  yok sayılacağı hakkında iyi bir analiz yapmamız gerekmektedir, çünkü hassas hesaplamalarda her değer çok önemlidir bunun için en uygun veriler ile anliz yapmamız gerekmektedir. 

# Evet şimdi sıra veri setindeki eksik (NaN) değerlerini çıkarıyoruz.
___
Eğer işleminiz acele ve eksik verilerin olup olmaması durumu çok önemli değil ise(ki hassas bir analiz yapmıyorsak) veri setimizin sonuçlarını etkilememesi için satır ve sütunlarda bulunan eksik verileri (NaN-Not a Number) tablomuzdan çıkarabiliriz. 
Evet eğer bu durumdan eminsek yani eksik verilerin analizimizi etkilemeyeceğini  biliyorsak  verileri tablodan çıkarmak için Python kütüphanelerinden 'pandas' kütüp hanesi bize bu konuda yadımcı olacak ve 'dropna()' (eksik verileri  veri setinden çıkar) komutunu kullanacağız. 

Gelin bu işlemi NLF veri setimiz üzerinde deneyelim !.

In [10]:
#Evet şimdi tüm satırlardaki eksik(NaN) değerlerini siliyoruz
nlf_data.dropna()

Ahaa, gördünüz mü neler oldu ? Bütün sütünlar yok oldu, acaba hepsi eksik(NaN) değerleri ile mi dolu idi ?😱 Bunun neden kaynaklandığını biliyoruz değilmi, çünkü tüm satırlarda en az bir tane eksik(NaN) değer mevcut imiş dolayısı ile tüm satırları silmiş olduk. Gelin biz bu durumu satırlar için değil de *sütünlar* için yapalım.

In [13]:
#Bir tane bile eksik değer olan tüm kolonları silelim
colomns_with_na_dropped = nlf_data.dropna(axis=1)
colomns_with_na_dropped.head()

In [18]:
#Şimdi bakalım ne kadar değer kayıbımız var :)
print("Orijinal veri setinin sütunları : %d \n" % nlf_data.shape[1])
print("Eksik verileri çıkarıldıktan sonraki sütun sayısı : %d" % colomns_with_na_dropped.shape[1])

Görüldüğü üzere verilerimizin bütük kısmı silindi ama şunu başardık, artık veri setimiz içirisinde hiç bir sütünda eksik değer kalmadı.

In [20]:
#sf_permits veri setinden bir tane bile eksik değer olan tüm kolonları silelim
colomns_with_na_dropped = sf_permits.dropna(axis=1)
colomns_with_na_dropped.head()

In [21]:
#Gelin birde sf_permits veri setimizdeki eksik değer olan sütunları silelim, ne dersiniz ?
print("Orginal veri setinin sutunları : %d \n" % sf_permits.shape[1])
print("Eksik değerler silindikten sonra sf_permits veri setindeki  sütün sayısı : %d" % colomns_with_na_dropped.shape[1])

# Evet bu aşamada boş değerler yerine otomatik olarak değerler ekleyeceğiz.
 ___
 
Bu bölümde ve bundan sonraki kısımlarda NLF veri setinden bazı örnekler alarak işlemlerimizi gerçekleştireceğiz.

In [23]:
#NFL veri setinden bir alt küme veri seti alıyoruz
subset_nlf_data = nlf_data.loc[:, 'EPA':'Season'].head()
subset_nlf_data

Evet bura da da pandas  bize yardım ediyor, 'Pandas'ın 'fillna()' fonksiyonunu kullanarak veri setinde eksik olan veriler yerine yeni veriler ekleyeceğiz.  
Aslında burada yapmak istediğim şey boş 'NaN' olan değerler yerine sıfır '0' yazmak, gelin bu işlemi beraber yapalım.

In [25]:
#Tablodaki NaN değerlerinin yerine hepsine sıfır'0' yazıyoruz.
subset_nlf_data.fillna(0)

Aynı zamanda daha fazla bilgi sahibi olabilirim  ve aynı değerden sonra  gelen değerlerle birlikte eksik değerleri alabilirim. (Bu, gözlemlerin bir çeşit mantıksal düzenleri olduğu veri kümeleri için çok anlam ifade eder.)

In [26]:
#Aynı sütunlardaki tüm NaN değerlerini  sıfır(0) ile değiştiriyoruz 
subset_nlf_data.fillna(method = 'bfill', axis = 0).fillna("0")

Daha fazla pratik için linkteki dökümandan faydalanabilirsiniz.  [bu derste daha fazla pratik yapabilirsiniz "Daha çok pratik için " ](https://www.kaggle.com/dansbecker/handling-missing-values). Yada öncelikle neden se_permits veri setindeki boş(eksik) verileri sıfır ile değiştirmeyesiniz  ?

In [ ]:
# Pratik ! sf_permits veri setindeki NaN değerlerini sıfır '0' ile değiştiriniz.
# Sonra Aynı sütunda bulunan NaN değerlerini sıfır ile değiştiriniz.

Bugünkü çalışma pratiğimiz bu kadar, eğer sorularınız olursa [forumdan](https://www.kaggle.com/questions-and-answers). sorabilir yada [bana](eaehesap@gmail.com) adresinden ulaşabilirsiniz. Çalışmalarınızda başarılar.


# Daha fazla pratik için !
___

Eğer daha fazla pratik yaparak deneyiminizi artırmak isterseniz ekteki linklerdeki durumları çözümlemeniz faydalı olabilir.

* [Eksik veriler ile Çalışma](https://www.kaggle.com/dansbecker/handling-missing-values)
[Datasets listing](https://www.kaggle.com/datasets).) 
